## Optimal controller -- maximum possible performance

This notebook uses Alice's matching code to determine the maximum possible performance

Inputs:

1. a picked peak file, with the peak *boundaries*
1. the length of time an MS1 scan takes
1. the length of time an MS2 scan takes
1. N (for top-N)
1. the minimum intensity for fragmentation

The code then creates a bipartite graph with two types of nodes:

1. scans (of either MS1 or MS2)
1. peaks

An edge is added between a scan and a peak *iff*:

1. the scan start time is within the start and end RT of the peak
1. the peak had an intensity >= the min intensity at the time of the last MS1 scan

i.e. this is showing the best possible top-N performance if, by magic, one could not fragment anything that didn't turn into a peak.

In [7]:
import sys
import os
import bisect

from mass_spec_utils.data_import.mzml import MZMLFile
from mass_spec_utils.data_import.mzmine import load_picked_boxes

In [8]:
pymzm_folder = '/Users/simon/git/pymzm'
# pymzm_folder = '/home/simon/git/pymzm'
vimms_folder = '/Users/simon/git/vimms'
# vimms_folder = '/home/simon/git/vimms/'


In [9]:
sys.path.append(pymzm_folder)
sys.path.append(vimms_folder)

Local package import

In [10]:
from vimms.Roi import Roi

## Define parameters

In [11]:
min_rt = 0 # start value
max_rt = 26*60 # end value
topn_time_dict = {1: 0.60,2:0.20} # how long do the different scan types take
N = 10 # top-N value
min_ms1_intensity = 1e5 # min intensity for fragmentation
data_folder = '/Users/simon/University of Glasgow/Vinny Davies - CLDS Metabolomics Project/Experimental_Results/20200715_TopN_vs_ROI/from_controllers/QCB'
pp_file_path = os.path.join(data_folder,'Fullscan_QCB_box.csv')
mzml_file_path = os.path.join(data_folder,'Fullscan_QCB.mzML')
verbose = True



### 1. Load the picked peak boxes

In [12]:
boxes = load_picked_boxes(pp_file_path)
print("Loaded {} picked peak boxes".format(len(boxes)))

Loaded 6278 picked peak boxes


### 2. Load the mzml file

In [13]:
mzfile  = MZMLFile(mzml_file_path)

Loaded 3697 scans


### 3. Set up the scan times

In [14]:
scan_levels = []
scan_start_times = []
time = 0
while True:
    # add an MS1
    scan_levels.append(1)
    scan_start_times.append(time)
    time += topn_time_dict[1]
    if time > max_rt:
        break
    # add N MS2
    for n in range(N):
        scan_levels.append(2)
        scan_start_times.append(time)
        time += topn_time_dict[2]
        if time > max_rt:
            break
print("Created {} scan times and levels".format(len(scan_levels)))

Created 6601 scan times and levels


### 4. Add ROIs to the peak boxes

In [15]:
# reset in case we run more than once
for box in boxes:
    box.roi = None
    
for i,scan in enumerate(mzfile.scans):
    if scan.ms_level == 2:
        continue # skip MS2 scans
    if verbose and i%100 == 0:
        print(i,len(mzfile.scans))
        
    scan_rt = scan.rt_in_seconds
    
    # find the boxes that are active at this scan
    ok_boxes = list(filter(lambda x: x.rt_range_in_seconds[0] <= scan_rt and x.rt_range_in_seconds[1] >= scan_rt,boxes))
    if len(ok_boxes) == 0:
        continue # no boxes now, onto the next scan
    
    mz_list,intensity_list = zip(*scan.peaks)
    for box in ok_boxes:
        
        min_idx = bisect.bisect_right(mz_list,box.mz_range[0])
        max_idx = bisect.bisect_right(mz_list,box.mz_range[1])
        sub_peaks = []
        for i in range(min_idx,max_idx):
            sub_peaks.append((mz_list[i],intensity_list[i]))
        if len(sub_peaks) > 0:
            sub_peaks.sort(key = lambda x: x[1],reverse = True) # sort by descending intensity
            peak_mz = sub_peaks[0][0]
            peak_intensity = sub_peaks[0][1]
            peak_rt = scan_rt

            if box.roi is None:
                box.roi = Roi(peak_mz,peak_rt,peak_intensity)
            else:
                box.roi.add(peak_mz,peak_rt,peak_intensity)
                
n_with_roi = 0
for box in boxes:
    if box.roi is not None:
        n_with_roi += 1
print("Of {} boxes, {} have roi".format(len(boxes),n_with_roi))

0 3697
100 3697
200 3697
300 3697
400 3697
500 3697
600 3697
700 3697
800 3697
900 3697
1000 3697
1100 3697
1200 3697
1300 3697
1400 3697
1500 3697
1600 3697
1700 3697
1800 3697
1900 3697
2000 3697
2100 3697
2200 3697
2300 3697
2400 3697
2500 3697
2600 3697
2700 3697
2800 3697
2900 3697
3000 3697
3100 3697
3200 3697
3300 3697
3400 3697
3500 3697
3600 3697
Of 6278 boxes, 6278 have roi


In [16]:
# a little test
box_pos = 2345
print(boxes[box_pos],boxes[box_pos].rt_range_in_seconds)
print(boxes[box_pos].roi.mz_list)

2346: [247.10794067382812, 247.11643981933594] [7.6004010712, 7.72969039975] [456.02406427200003, 463.781423985]
[247.11607360839844, 247.10842895507812, 247.10824584960938, 247.11618041992188, 247.11611938476562, 247.11643981933594, 247.1160888671875, 247.11622619628906, 247.10818481445312, 247.10826110839844, 247.10797119140625, 247.1082305908203, 247.10801696777344, 247.1080322265625, 247.108154296875, 247.10812377929688, 247.1079559326172, 247.108154296875]


### 5. Make the graph edges

1. Loop over the boxes
1. Find all MS1 scans at which the intensity exceeds the minmum
1. Add edges to all ms2 scans within the block

Note:
 - This method ignores the intensity *at the MS2 scan time*


In [17]:
def get_intensity(roi,rt):
    if rt < roi.rt_list[0] or rt > roi.rt_list[-1]:
        return 0
    else:
        pos = bisect.bisect_right(roi.rt_list,rt)
        before_pos = pos - 1
        after_pos = pos
        prop = (rt - roi.rt_list[before_pos])/(roi.rt_list[after_pos] - roi.rt_list[before_pos])
        return roi.intensity_list[before_pos] + prop*(roi.intensity_list[after_pos] - roi.intensity_list[before_pos])

edges = []

for box in boxes:
    peak_id = box.peak_id
    rt_start = box.rt_range_in_seconds[0]
    rt_end = box.rt_range_in_seconds[1]
    spo = bisect.bisect_right(scan_start_times,rt_start)
    can_fragment = False # until we see an MS1
    while spo < len(scan_start_times) and scan_start_times[spo] < rt_end:
        if scan_levels[spo] == 1:
            if get_intensity(box.roi,scan_start_times[spo]) >= min_ms1_intensity:
                can_fragment = True
            else:
                can_fragment = False
        if scan_levels[spo] == 2:
            if can_fragment:
                edges.append((spo,peak_id,box))
        spo += 1
print("{} possible edges found".format(len(edges)))

23842 possible edges found


In [19]:
# add AM code here


In [ ]:
import csv
orig_edges = []
with open('/home/simon/git/vimms/experimental/graph_optimal/qca_5e3_edges.csv','w') as f:
    writer = csv.writer(f)
    for scan,peak in edges:
        writer.writerow(['S{}'.format(scan),'B{}'.format(peak)])
        orig_edges.append(['B{}'.format(peak),'S{}'.format(scan)])

In [ ]:
edges = set(edges)

In [ ]:
print(len(edges))

Check the initial results

In [ ]:
res_file = '/home/simon/git/vimms/experimental/graph_optimal/qcb_edgesResults.txt'
matches = []
with open(res_file,'r') as f:
    reader = csv.reader(f,delimiter = ',')
    for line in reader:
        if len(line) < 2:
            continue
        if '.' in line[0]:
            continue
        matches.append(line)
unique_boxes = set([x[0] for x in matches])
dashes = list(filter(lambda x: x[1] == '-',matches))
matches = list(filter(lambda x: not x[1] == '-',matches))
print(len(unique_boxes))
print(len(dashes))

original_unique_boxes = set((x[0] for x in orig_edges))
print(len(original_unique_boxes))

In [ ]:
unique_boxes = set([x[0] for x in matches])
unique_scans = set([x[1] for x in matches])
print(len(unique_boxes))
print(len(unique_scans))

In [ ]:
# make a schedule
schedule = []
box_dict = {b.peak_id:b for b in boxes}
count = 0
for i,(level,time) in enumerate(zip(scan_levels,scan_start_times)):
    s_id = 'S{}'.format(i)
    if level == 1:
        schedule.append(('MS1',None,time,None,pp_file))
        match = list(filter(lambda x: x[1] == s_id,matches))
        assert len(match) == 0
    else:
        # find it in the matching
        match = list(filter(lambda x: x[1] == s_id,matches))
        assert len(match) <= 1
        if len(match) == 0:
            schedule.append(('MS2',-1,time,None,pp_file)) # set precursor to -1 to indicate empty
        else:
            count += 1
            box_id = int(match[0][0][1:])
            box = box_dict[box_id]
            mz = box.mz
            schedule.append(['MS2',mz,time,box_id,pp_file])
print(count)

In [ ]:
with open('/home/simon/git/vimms/experimental/graph_optimal/qcb_schedule.csv','w') as f:
    writer = csv.writer(f)
    for row in schedule:
        writer.writerow(row)

# score by distance from apex

- Compute a score from 1 to 10 where 10 is as close to the apex as possible and then decay as the scans get further away

In [ ]:
# Match the boxes back to the original signal

In [ ]:
from ms2_matching import MZMLFile

In [ ]:
# loop over scans, adding intensity values to the boxes

In [ ]:
sub_boxes = [boxes[0]]
print(sub_boxes)

In [ ]:
print(boxes[0].roi)
print(boxes[0])

In [ ]:
import pylab as plt
%matplotlib inline
plt.figure(figsize = (10,10))
plt.plot(boxes[1].roi.rt_list,boxes[1].roi.intensity_list,'ro')
plt.xlim([520,530])
plt.plot(527,get_intensity(boxes[1].roi,527),'bo')
plt.ylim([0,6e6])

In [ ]:
import bisect


In [ ]:
count = 0
total = 0
no_roi = 0
for level,precursor,time,box_id,_ in schedule:
    if level == 'MS2' and precursor > -1:
        total +=1
        box = box_dict[box_id]
        if not box.roi is None:
            intensity = get_intensity(box.roi,time)
            if intensity >= 5e3:
                count += 1
        else:
            no_roi += 1
print(count,total,no_roi)